# Import Library & Install Package

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/DIV TEKNIK INFORMATIKA/Pemrograman Sistem Cerdas - Bu Intan/Projek Sentiment Analysis/'

/content/drive/MyDrive/DIV TEKNIK INFORMATIKA/Pemrograman Sistem Cerdas - Bu Intan/Projek Sentiment Analysis


In [ ]:
!pip install dataframe-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.5/398.5 kB 29.2 MB/s eta 0:00:00


In [ ]:
!pip install varname

In [ ]:
!pip install keras-metrics

In [ ]:
!pip install h5py

In [ ]:
!pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import keras_metrics as km
import pickle
import keras

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from varname import nameof

nltk.download('punkt')
from nltk.tokenize import word_tokenize

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Load Dataset

In [ ]:
path = "review.csv"
ds = pd.read_csv(path, sep=",", header=[0], encoding="UTF-8")
ds.head()

,content,score
0,Error,1
1,"Tulisan ""page unavailable"". Mengganti profil s...",1
2,Usernya banyak yang toxic,1
3,"Setiap membuka aplikasinya, aplikasinya selalu...",1
4,"Jujur sih aplikasi ini ribet banget cuy,sepert...",1


In [ ]:
dsCont = ds['content']
dsCont

0                                                  Error
1      Tulisan "page unavailable". Mengganti profil s...
2                              Usernya banyak yang toxic
3      Setiap membuka aplikasinya, aplikasinya selalu...
4      Jujur sih aplikasi ini ribet banget cuy,sepert...
                             ...                        
391                                                   👍👍
392                                            I like it
393    ini tempat review buku. lihat penilaian orang ...
394    Saya rasa bagus sekali jika ada aplikasi seper...
395                                                Siiip
Name: content, Length: 396, dtype: object

#Case Folding

In [ ]:
dsCf = dsCont.str.lower()
dsCf.head()

0                                                error
1    tulisan "page unavailable". mengganti profil s...
2                            usernya banyak yang toxic
3    setiap membuka aplikasinya, aplikasinya selalu...
4    jujur sih aplikasi ini ribet banget cuy,sepert...
Name: content, dtype: object

# Filtering

In [ ]:
#Filtering
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|
\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))\n''',
 " ", content) for content in dsCf]
filtering_cont = [re.sub(r'\(cont\)'," ", content)for content in filtering_url]
filtering_punctuation = [re.sub(r'[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', content) for content in filtering_cont]
filtering_tagger = [re.sub(r'#([^\s]+)', '', content) for content in filtering_punctuation]
filtering_numeric = [re.sub(r'\d+', ' ', content) for content in filtering_tagger]
dsFilter = pd.Series(filtering_numeric)
dsFilter.head()

0                                                error
1    tulisan  page unavailable   mengganti profil s...
2                            usernya banyak yang toxic
3    setiap membuka aplikasinya  aplikasinya selalu...
4    jujur sih aplikasi ini ribet banget cuy sepert...
dtype: object

#Tokenization

In [ ]:
#Tokenization
dsToken = [word_tokenize(line) for line in dsFilter]
print(dsToken)

[['error'], ['tulisan', 'page', 'unavailable', 'mengganti', 'profil', 'sampai', 'refresh', 'akun', 'diweb', 'dan', 'apk', 'tidak', 'bisa', 'bintang', 'satu'], ['usernya', 'banyak', 'yang', 'toxic'], ['setiap', 'membuka', 'aplikasinya', 'aplikasinya', 'selalu', 'keluar', 'sendiri', 'malah', 'selalu', 'keluar', 'saat', 'sya', 'sedang', 'mencari', 'buku'], ['jujur', 'sih', 'aplikasi', 'ini', 'ribet', 'banget', 'cuy', 'sepertinya', 'bukan', 'aplikasi', 'untuk', 'membaca', 'sih', 'delete', 'this', 'aplication'], ['seringkali', 'buku', 'tidak', 'bisa', 'dibaca'], ['punyaku', 'kok', 'beda', 'dari', 'yang', 'lainn', 'padahal', 'punyaku', 'bukan', 'pick', 'me', '\u2060｡\u2060•́\u2060︿\u2060•̀\u2060｡\u2060'], ['aplikasi', 'nya', 'ngk', 'praktis'], ['aplikasi', 'gak', 'guna'], ['kirain', 'ada', 'kumpulan', 'buku', 'di', 'sini', 'ternyata', 'cuma', 'judul'], ['banyak', 'yg', 'kasih', 'spoiler', 'tanpa', 'tw'], ['ca', "n't", 'see', 'any', 'community', 'reviews'], ['nyesel', 'aja', 'donlod', 'di', '

#StopWord Remover

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
factory = StopWordRemoverFactory()
ind_stopword = factory.get_stop_words()

def stopword(line):
    temp = list()
    for word in line:
       if (word not in ind_stopword):
          if(len(word)>3):
              temp.append(word)
    return temp

dsPenting = [stopword (line) for line in dsToken]
print(dsPenting)

[['error'], ['tulisan', 'page', 'unavailable', 'mengganti', 'profil', 'refresh', 'akun', 'diweb', 'bintang', 'satu'], ['usernya', 'banyak', 'toxic'], ['membuka', 'aplikasinya', 'aplikasinya', 'selalu', 'keluar', 'sendiri', 'malah', 'selalu', 'keluar', 'sedang', 'mencari', 'buku'], ['jujur', 'aplikasi', 'ribet', 'banget', 'sepertinya', 'bukan', 'aplikasi', 'membaca', 'delete', 'this', 'aplication'], ['seringkali', 'buku', 'dibaca'], ['punyaku', 'beda', 'lainn', 'padahal', 'punyaku', 'bukan', 'pick', '\u2060｡\u2060•́\u2060︿\u2060•̀\u2060｡\u2060'], ['aplikasi', 'praktis'], ['aplikasi'], ['kirain', 'kumpulan', 'buku', 'sini', 'ternyata', 'cuma', 'judul'], ['banyak', 'kasih', 'spoiler'], ['community', 'reviews'], ['nyesel', 'donlod', 'buka', 'cuman', 'judul', 'bukak', 'baca'], ['cant', 'continue', 'login', 'with', 'facebook', 'account', 'when', 'want', 'login', 'email', 'also'], ['digunain', 'cuma', 'reload', 'mulu', 'padahal', 'udah', 'update', 'versi', 'terbaru', 'signal', 'bagus'], ['hii

#Slang Word Convertion

In [ ]:
path_slang = open("kamus kata baku 1.csv")
slang = pd.read_csv(path_slang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(slang[0]):
    indexslang = list(slang[0]).index(word)
    return slang[1][indexslang]
  else:
    return word

ulasan_formal= []
for data in dsPenting:
  ds_clean = [replaceSlang(word) for word in data]
  ulasan_formal.append(ds_clean)
#len_ulasan_formal = len(ulasan_formal)
print(ulasan_formal)
#len_ulasan_formal

[['error'], ['tulisan', 'page', 'unavailable', 'mengganti', 'profil', 'refresh', 'akun', 'diweb', 'bintang', 'satu'], ['usernya', 'banyak', 'toxic'], ['membuka', 'aplikasinya', 'aplikasinya', 'selalu', 'keluar', 'sendiri', 'malah', 'selalu', 'keluar', 'sedang', 'mencari', 'buku'], ['jujur', 'aplikasi', 'ribet', 'banget', 'sepertinya', 'bukan', 'aplikasi', 'membaca', 'delete', 'this', 'aplication'], ['seringkali', 'buku', 'dibaca'], ['punyaku', 'beda', 'lainn', 'padahal', 'punyaku', 'bukan', 'pick', '\u2060｡\u2060•́\u2060︿\u2060•̀\u2060｡\u2060'], ['aplikasi', 'praktis'], ['aplikasi'], ['diduga', 'kumpulan', 'buku', 'sini', 'ternyata', 'cuma', 'judul'], ['banyak', 'kasih', 'spoiler'], ['community', 'reviews'], ['menyesal', 'donlod', 'buka', 'cuma', 'judul', 'buka', 'baca'], ['cant', 'continue', 'login', 'with', 'facebook', 'account', 'when', 'want', 'login', 'email', 'also'], ['digunain', 'cuma', 'reload', 'terus', 'padahal', 'sudah', 'update', 'versi', 'terbaru', 'signal', 'bagus'], ['h

# Stemming

In [ ]:
factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      #if(word not in white_list):
        word = ind_stemmer.stem(word)
      #if(len(word)>3):
        temp.append(word)
    return temp

kata_dasar = [stemmer (line) for line in ulasan_formal]
print(kata_dasar)

[['error'], ['tulis', 'page', 'unavailable', 'ganti', 'profil', 'refresh', 'akun', 'diweb', 'bintang', 'satu'], ['usernya', 'banyak', 'toxic'], ['buka', 'aplikasi', 'aplikasi', 'selalu', 'keluar', 'sendiri', 'malah', 'selalu', 'keluar', 'sedang', 'cari', 'buku'], ['jujur', 'aplikasi', 'ribet', 'banget', 'seperti', 'bukan', 'aplikasi', 'baca', 'delete', 'this', 'aplication'], ['seringkali', 'buku', 'baca'], ['punya', 'beda', 'lainn', 'padahal', 'punya', 'bukan', 'pick', ''], ['aplikasi', 'praktis'], ['aplikasi'], ['duga', 'kumpul', 'buku', 'sini', 'nyata', 'cuma', 'judul'], ['banyak', 'kasih', 'spoiler'], ['community', 'reviews'], ['sesal', 'donlod', 'buka', 'cuma', 'judul', 'buka', 'baca'], ['cant', 'continue', 'login', 'with', 'facebook', 'account', 'when', 'want', 'login', 'email', 'also'], ['digunain', 'cuma', 'reload', 'terus', 'padahal', 'sudah', 'update', 'versi', 'baru', 'signal', 'bagus'], ['hiiit', 'force', 'close'], ['jelek', 'baca'], ['masuk', 'mental', 'mental', 'mulu'], ['

In [ ]:
import json
jsons= json.dumps(kata_dasar)
jsonFile = open("data_preprocessing.json", "w")
jsonFile.write(jsons)
jsonFile.close()

fileObject = open("data_preprocessing.json", "r")
jsonContent = fileObject.read()
ds_preprocess = json.loads(jsonContent)
ds_preprocess

[['error'],
 ['tulis',
  'page',
  'unavailable',
  'ganti',
  'profil',
  'refresh',
  'akun',
  'diweb',
  'bintang',
  'satu'],
 ['usernya', 'banyak', 'toxic'],
 ['buka',
  'aplikasi',
  'aplikasi',
  'selalu',
  'keluar',
  'sendiri',
  'malah',
  'selalu',
  'keluar',
  'sedang',
  'cari',
  'buku'],
 ['jujur',
  'aplikasi',
  'ribet',
  'banget',
  'seperti',
  'bukan',
  'aplikasi',
  'baca',
  'delete',
  'this',
  'aplication'],
 ['seringkali', 'buku', 'baca'],
 ['punya', 'beda', 'lainn', 'padahal', 'punya', 'bukan', 'pick', ''],
 ['aplikasi', 'praktis'],
 ['aplikasi'],
 ['duga', 'kumpul', 'buku', 'sini', 'nyata', 'cuma', 'judul'],
 ['banyak', 'kasih', 'spoiler'],
 ['community', 'reviews'],
 ['sesal', 'donlod', 'buka', 'cuma', 'judul', 'buka', 'baca'],
 ['cant',
  'continue',
  'login',
  'with',
  'facebook',
  'account',
  'when',
  'want',
  'login',
  'email',
  'also'],
 ['digunain',
  'cuma',
  'reload',
  'terus',
  'padahal',
  'sudah',
  'update',
  'versi',
  'baru',

# Vectorizer TF/DF

In [ ]:
vect = TfidfVectorizer()
reviews2 = ["".join(r) for r in ds_preprocess]
vect_tfidf = vect.fit_transform(reviews2)
vect_tfidf2 = vect_tfidf.toarray()
vect_tfidf2.shape

(396, 336)

#Modeling

In [ ]:
labels = ds['score']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(vect_tfidf2, labels, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [ ]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(vect_tfidf2, ds['score'])

LinearSVC()

#Prediction

In [ ]:
new_text = ["Aplikasi Bagus, Keren"]
new_tfidf = vect.transform(new_text).toarray()
prediction = svm_model.predict(new_tfidf)
if prediction == 5:
  print('Hasil Sentimen Analisis : Positif')
elif prediction == 1:
  print('Negatif')

Positif


#Saving Model

In [ ]:
import pickle

drive_path = '/content/drive/MyDrive/DIV TEKNIK INFORMATIKA/Pemrograman Sistem Cerdas - Bu Intan/Projek Sentiment Analysis/'

# Simpan model SVM ke dalam file .pkl menggunakan pickle
model_filename = 'sentimentSVM.pkl'
with open(drive_path + model_filename, 'wb') as model_file:
    pickle.dump(svm_model, model_file)

print(f"Model telah disimpan sebagai {model_filename}.")

Model telah disimpan sebagai sentimentSVM.pkl.


# Evaluasi

In [ ]:
#Accuracy
accuracy = svm_model.score(x_test, y_test)
print("Akurasi Model: {:.2f}%".format(accuracy * 100))

Akurasi Model: 97.50%


In [ ]:
#Recall
y_pred = svm_model.predict(x_test)
recall = recall_score(y_test, y_pred, average='weighted')
print(f'Recall: {recall * 100:.2f}%')

Recall: 97.50%


In [ ]:
#F1 Score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {f1 * 100:.2f}%')

F1 Score: 97.51%
